In [1]:
import numpy as np
from jax import jit, vmap, debug
import jax.numpy as jnp
import pandas as pd
import seaborn as sns
import os,sys,re
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
from itertools import cycle, islice
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large',
         'image.cmap':'jet'}
plt.rcParams.update(params)
import matplotlib.offsetbox
props = dict(boxstyle='round',edgecolor="w",facecolor="w", alpha=0.5)
from tqdm import tqdm
from EmuLP import Analysis, Template, Extinction, Galaxy, Filter

2023-10-20 16:08:15.715477: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:497] The NVIDIA driver's CUDA version is 11.3 which is older than the ptxas CUDA version (11.4.152). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [ ]:
summary_props = pd.read_pickle('FORS2_spectra_classif_properties.pkl')
df_sl_spec = pd.read_pickle('FORS2-SL_spectra_for_templates.pkl')

In [ ]:
_gal_seds = 'FORS2_ColorSelect_batch5'
sed_dir = os.path.join(LEPHAREDIR, 'sed/GAL', _gal_seds)
if not os.path.isdir(sed_dir) : os.makedirs(sed_dir)
list_file_name = f'SED_FORS2_ColorSelect_SL_v5'

In [ ]:
if False:
    with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'w') as _l:
        _l.write(f'# FORS2-derived templates selected and averaged regarding their position in the 4D color space\n')
    if use_average:
        for _col in df_4D_col_avg.columns[1:]:
            _arr = np.column_stack((df_4D_col_avg['Wavelength'], df_4D_col_avg[_col]))
            np.savetxt(os.path.join(sed_dir,f'{_col}.sed'), _arr, fmt=('%10.1f', '%10.8f'), delimiter=" ")
            with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'a') as _l:
                _l.write(_gal_seds+f'/{_col}.sed\n')
    else:
        for _spec in df_fors2SL_prop["Spectrum"][_mask]:
            _arr = np.column_stack((df_fors2SL_spec['Wavelength'], df_fors2SL_spec[_spec]))
            np.savetxt(os.path.join(sed_dir,f'{_spec}.sed'), _arr, fmt=('%10.1f', '%10.8f'), delimiter=" ")
            with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'a') as _l:
                _l.write(_gal_seds+f'/{_spec}.sed\n')

In [ ]:
if False:
    list_file_name = list_file_name+'_normed'
    with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'w') as _l:
        _l.write(f'# FORS2-derived templates selected and averaged regarding their position in the 4D color space. Normalised by the integral between {lambdaMin} and {lambdaMax} angstrom.\n')
    if use_average:
        for _col in df_4D_col_avg_normed.columns[1:]:
            _arr = np.column_stack((df_4D_col_avg_normed['Wavelength'], df_4D_col_avg_normed[_col]))
            np.savetxt(os.path.join(sed_dir,f'{_col}.sed'), _arr, fmt=('%10.1f', '%10.8f'), delimiter=" ")
            with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'a') as _l:
                _l.write(_gal_seds+f'/{_col}.sed\n')
    else:
        for _spec in df_fors2SL_prop["Normalized spectrum"][_mask]:
            _arr = np.column_stack((df_fors2SL_spec_normed['Wavelength'], df_fors2SL_spec_normed[_spec]))
            np.savetxt(os.path.join(sed_dir,f'{_spec}.sed'), _arr, fmt=('%10.1f', '%10.8f'), delimiter=" ")
            with open(os.path.join(LEPHAREWORK, list_file_name+'.list'), 'a') as _l:
                _l.write(_gal_seds+f'/{_spec}.sed\n')